In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
#for slicing error message
import sys, os

In [ ]:
x = np.random.sample((100,2))
print("First  5 pairs in x: \n", x[:5], "\n" )
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

In [ ]:
# using two numpy arrays
features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
print("First  5 features in features: \n", features[:5], "\n" )
print("First  5 labels in labels: \n", labels[:5], "\n" )

dataset = tf.data.Dataset.from_tensor_slices((features,labels))

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print(sess.run(el))

In [ ]:
# using a tensor
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([100, 2]))
print("dataset shape: ", np.shape(dataset))
print("dataset: ", (dataset))

iter = dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))

In [ ]:
# using a placeholder
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)

data = np.random.sample((100,2))
print("First  5 pairs in data: \n", data[:5], "\n" )

iter = dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer, feed_dict={ x: data })
    print(sess.run(el))

In [ ]:
# from generator
sequence = np.array([[[1]],[[2],[3]],[[3],[4],[5]]])

def generator():
    for el in sequence:
        yield el

dataset = tf.data.Dataset().batch(1).from_generator(generator,
                                           output_types= tf.int64, 
                                           output_shapes=(tf.TensorShape([None, 1])))

iter = dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el),"\n")
    print(sess.run(el),"\n")
    print(sess.run(el),"\n")


In [ ]:
# initializable iterator to switch between data
EPOCHS = 10

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y))

train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.array([[1,2]]), np.array([[0]]))

iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

with tf.Session() as sess:
#     initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
    for _ in range(EPOCHS):
        sess.run([features, labels])
#     switch to test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    print(sess.run([features, labels]))

    
    

In [ ]:
# Reinitializable iterator to switch between Datasets
#EVIDENCE: Can re-read the same data
NO_OF_BATCHES = 10
DATA_ITEMS = 10
NO_OF_EPOCHS = 2
# making fake data using numpy
train_data = (np.random.sample((DATA_ITEMS,2)), np.random.sample((DATA_ITEMS,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))

# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)

# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)

assert(train_dataset.output_types == test_dataset.output_types)
assert(train_dataset.output_shapes == test_dataset.output_shapes)

features, labels = iter.get_next()

# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

with tf.Session() as sess:
    for epoch in range(NO_OF_EPOCHS):
        print("Epoch: ", epoch)
        sess.run(train_init_op) # switch to train dataset
        for _ in range(NO_OF_BATCHES):
            print("Train: ",sess.run([features, labels]))
        print()
        
        COMMENT_OUT = True
        if(not COMMENT_OUT):
            sess.run(train_init_op) # switch to train dataset
            for _ in range(NO_OF_BATCHES):
                print("Train: ",sess.run([features, labels]))
            print()
            
        sess.run(test_init_op) # switch to val dataset
        for _ in range(NO_OF_BATCHES):
            print("Test: ",sess.run([features, labels]))

    
    

In [ ]:
# feedable iterator to switch between iterators
EPOCHS = 10
# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))
# create placeholder
x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices((x,y))
test_dataset = tf.data.Dataset.from_tensor_slices((x,y))
# create the iterators from the dataset
train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()
# same as in the doc https://www.tensorflow.org/programmers_guide/datasets#creating_an_iterator
handle = tf.placeholder(tf.string, shape=[])
iter = tf.data.Iterator.from_string_handle(
    handle, train_dataset.output_types, train_dataset.output_shapes)
next_elements = iter.get_next()

with tf.Session() as sess:
    train_handle = sess.run(train_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())
    
    # initialise iterators. In our case we could have used the 'one-shot' iterator instead,
    # and directly feed the data insted the Dataset.from_tensor_slices function, but this
    # approach is more general
    sess.run(train_iterator.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
    sess.run(test_iterator.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    
    for _ in range(EPOCHS):
        x,y = sess.run(next_elements, feed_dict = {handle: train_handle})
        print(x, y)
        
    x,y = sess.run(next_elements, feed_dict = {handle: test_handle})
    print(x,y)

In [ ]:
# BATCHING
BATCH_SIZE = 4
x = np.random.sample((100,2))
print("First  8 pairs in x: \n", x[:8], "\n" )

# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x).batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print("First  2 batches (each of 4 pairs) in dataset:")
    print(sess.run(el));    print(sess.run(el))

In [ ]:
# REPEAT
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.repeat() #<<<<<========================
                                                        #^
iter = dataset.make_one_shot_iterator()                 #^
el = iter.get_next()                                    #^
                                                        #^
#with tf.Session() as sess:                              #^
#     this will run forever #==Because of ==========>>>>>>
#    while True:
#        print(sess.run(el))

In [ ]:
# MAP
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
print("Type of dataset before map(): ", dataset)

dataset = dataset.map(lambda x: x*2)
print("Type of dataset after map(): ", dataset)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
#     this will run forever.  
#RM: NOT correct. There is no dataset = dataset.repeat() here.
        for i in range(len(x)):
            try:
                print("i: ", i, ". x[i]: ", sess.run(el))
                print("i: ", i, ", x[i]: ", sess.run(el))
                print("i: ", i, ", x[i]: ", sess.run(el))
            except:
                """
                from:
                https://stackoverflow.com/questions/1278705/python-when-i-catch-an-exception-how-do-i-get-the-type-file-and-line-number
                """
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print("i: ", i, "; Exception thrown as iterator is at the end. \n\tException Type: ", exc_type,\
                      "; \n\tIn File: ", fname, "; ", \
                      "\n\tAt Line No: ",exc_tb.tb_lineno)      

In [ ]:
# SHUFFLE
BATCH_SIZE = 4
x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    try:
        print(sess.run(el), "\n")
        print(sess.run(el), "\n")
    except:
        """
        from:
        https://stackoverflow.com/questions/1278705/python-when-i-catch-an-exception-how-do-i-get-the-type-file-and-line-number
        """
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("Exception thrown as iterator is at the end. \n\tException Type: ", exc_type,\
              "; \n\tIn File: ", fname, "; ", \
              "\n\tAt Line No: ",exc_tb.tb_lineno)  
        
#With new session there should not be any problem        
with tf.Session() as sess:
    print("But With new session there should not be any problem.\n", sess.run(el), "\n")


In [ ]:
# how to pass the value to a model
EPOCHS = 10
BATCH_SIZE = 16
# using two numpy arrays
features, labels = (np.array([np.random.sample((100,2))]), 
                    np.array([np.random.sample((100,1))]))

dataset = tf.data.Dataset.from_tensor_slices((features,labels)).repeat().batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
x, y = iter.get_next()

# make a simple model
net = tf.layers.dense(x, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, y) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(EPOCHS):
        _, loss_value = sess.run([train_op, loss])
        print("Iter: {}, Loss: {:.4f}".format(i, loss_value))

In [ ]:
# Wrapping all together -> Switch between train and test set using Initializable iterator
NO_OF_EPOCHS = 10
######################
#RM
NoOfTrainingSamples = 100
NoOfTestSamples=20
INPUT_SHAPE = [None, 2] #Input data. 2-D Array of features
OUTPUT_SHAPE = [None, 1] #Output Classes. 1-D array of Labels
BATCH_SIZE = 10
#######################
# create a placeholder to dynamically switch between batch sizes
batch_size = tf.placeholder(tf.int64)

x, y = tf.placeholder(tf.float32, shape=INPUT_SHAPE), \
                tf.placeholder(tf.float32, shape=OUTPUT_SHAPE)
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat()

iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()
############################
# make a simple model
net = tf.layers.dense(features, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, labels) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

No_Of_Batches = NoOfTrainingSamples//BATCH_SIZE

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # using two numpy arrays
    train_data = (np.random.sample((NoOfTrainingSamples,INPUT_SHAPE[1])), \
                  np.random.sample((NoOfTrainingSamples,OUTPUT_SHAPE[1])))
    test_data = (np.random.sample((NoOfTestSamples,INPUT_SHAPE[1])), \
                 np.random.sample((NoOfTestSamples,OUTPUT_SHAPE[1])))

    # initialise iterator with train data
    #See https://www.tensorflow.org/api_docs/python/tf/data/Dataset#make_initializable_iterator
    # The initializer property returns the tf.operation that is run. It will initialize the iterator.
    # See https://www.tensorflow.org/api_docs/python/tf/data/Iterator#initializer
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    ListOfFeatureLabelPairs = sess.run(iter.get_next())
    print(ListOfFeatureLabelPairs)
    print("List Of features: ", ListOfFeatureLabelPairs[0])
    print("List of labels: ", ListOfFeatureLabelPairs[1])
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # using two numpy arrays
    train_data = (np.random.sample((NoOfTrainingSamples,INPUT_SHAPE[1])), \
                  np.random.sample((NoOfTrainingSamples,OUTPUT_SHAPE[1])))
    test_data = (np.random.sample((NoOfTestSamples,INPUT_SHAPE[1])), \
                 np.random.sample((NoOfTestSamples,OUTPUT_SHAPE[1])))

    # initialise iterator with train data
    #See https://www.tensorflow.org/api_docs/python/tf/data/Dataset#make_initializable_iterator
    # The initializer property returns the tf.operation that is run. It will initialize the iterator.
    # See https://www.tensorflow.org/api_docs/python/tf/data/Iterator#initializer
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    print('Training...')
    for epoch in range(NO_OF_EPOCHS):
        tot_loss = 0
        for _ in range(No_Of_Batches + 1):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Epoch No: {}, Loss: {:.4f}".format(epoch, tot_loss / No_Of_Batches))
        
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1], batch_size: test_data[0].shape[0]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))

    ##########################################################
    # Repeat above to prove that the iterator does not stall when it comes to the end
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    print('Training Again...')
    for epoch in range(NO_OF_EPOCHS):
        tot_loss = 0
        for _ in range(No_Of_Batches):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Epoch No: {}, Loss: {:.4f}".format(epoch, tot_loss / No_Of_Batches))
        
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1], batch_size: test_data[0].shape[0]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))

    ##########################################################
    # Repeat yet again to prove that the iterator does not stall when it comes to the end
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    print('Training yet Again...')
    for epoch in range(NO_OF_EPOCHS):
        tot_loss = 0
        for _ in range(No_Of_Batches):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Epoch No: {}, Loss: {:.4f}".format(epoch, tot_loss / No_Of_Batches))
        
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1], batch_size: test_data[0].shape[0]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))

    ##########################################################
    # Repeat yet again to prove that the iterator does not stall when it comes to the end
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    print('Training yet Again...')
    for epoch in range(NO_OF_EPOCHS):
        tot_loss = 0
        for _ in range(No_Of_Batches):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Epoch No: {}, Loss: {:.4f}".format(epoch, tot_loss / No_Of_Batches))
        
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1], batch_size: test_data[0].shape[0]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))


In [ ]:
# Wrapping all together -> Switch between train and test set using Reinitializable iterator
EPOCHS = 10
######################
#RM
NoOfTrainingSamples = 100
NoOfTestSamples=20
#######################
# create a placeholder to dynamically switch between batch sizes
batch_size = tf.placeholder(tf.int64)

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
train_dataset = tf.data.Dataset.from_tensor_slices((x,y)).batch(batch_size).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((x,y)).batch(batch_size) # always batch even 
                                                                #if you want to one shot it
# using two numpy arrays
train_data = (np.random.sample((NoOfTrainingSamples,2)), np.random.sample((NoOfTrainingSamples,1)))
test_data = (np.random.sample((NoOfTestSamples,2)), np.random.sample((NoOfTestSamples,1)))

# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
features, labels = iter.get_next()

# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

# make a simple model
net = tf.layers.dense(features, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, labels) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

n_batches = int(NoOfTrainingSamples/BATCH_SIZE)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # initialise iterator with train data
    sess.run(train_init_op, feed_dict = {x : train_data[0], y: train_data[1], batch_size: 16})
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(n_batches):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}".format(i, tot_loss / n_batches))
    # initialise iterator with test data
    sess.run(test_init_op, feed_dict = {x : test_data[0], y: test_data[1], batch_size:len(test_data[0])})
    print('Test Loss: {:4f}'.format(sess.run(loss)))


In [ ]:
# load a csv
CSV_PATH = './tweets.csv'
dataset = tf.contrib.data.make_csv_dataset(CSV_PATH, batch_size=32, shuffle=False) 
#RM: Original code default value of shuffle (True) was used. With shuffle set to False we read the CSV
#file, row-by-row.

iter = dataset.make_one_shot_iterator()
next = iter.get_next()
#print("iter.get_next(): ", next, "\n") # next is a dict with key=columns names and value=column data

#################################################
#
inputs, labels = next['text'], next['sentiment']
#################################################

with  tf.Session() as sess:
    print(sess.run([inputs,labels]), "\n")
    print(sess.run([inputs,labels]), "\n")

In [ ]:
log_time = {}
# copied form https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d
def how_much(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__)
            kw['log_time'][name] = (te - ts)
            
        return result
    return timed

In [ ]:
# benchmark
import time
DATA_SIZE = 5000
DATA_SHAPE = ((32,32),(20,))
BATCH_SIZE = 64 
N_BATCHES = DATA_SIZE // BATCH_SIZE
EPOCHS = 10

test_size = (DATA_SIZE//100)*20 

DUMMY = -1

train_shape = (DATA_SHAPE)
print("DATA_SHAPE:",train_shape)
train_shape = (DATA_SHAPE[0])
print("DATA_SHAPE[0]:",train_shape)
train_shape = ((DUMMY, *DATA_SHAPE))
print("DUMMY, *DATA_SHAPE:",train_shape)
train_shape = ((DUMMY, (*DATA_SHAPE))[0])
print("DUMMY, (*DATA_SHAPE))[0]:",train_shape)
train_shape = ((DUMMY, (*DATA_SHAPE)))
print("DUMMY, (*DATA_SHAPE):",train_shape, "\n")


train_shape = ((DATA_SIZE, *DATA_SHAPE[0]),(DATA_SIZE, *DATA_SHAPE[1]))
test_shape = ((test_size, *DATA_SHAPE[0]),(test_size, *DATA_SHAPE[1]))
print(train_shape, test_shape)

In [ ]:
train_data = (np.random.sample(train_shape[0]), np.random.sample(train_shape[1]))
test_data = (np.random.sample(test_shape[0]), np.random.sample(test_shape[1])) 

# used to keep track of the methodds
log_time = {}

tf.reset_default_graph()
#sess = tf.InteractiveSession() #This needs to be closed below. Replaced it with "with tf.Session() as sess" in 
                                #each of the functions 

input_shape = [None, *DATA_SHAPE[0]] # [None, 64, 64, 3]
output_shape = [None,*DATA_SHAPE[1]] # [None, 20]
print("input_shape: {} output_shape: {}".format(input_shape, output_shape))

x, y = tf.placeholder(tf.float32, shape=input_shape), tf.placeholder(tf.float32, shape=output_shape)

@how_much
def one_shot(**kwargs):
    print('\none_shot:')
    
    #The datasets - both train and test - are loaded
    #Two data sets. Two iterators. One for each
    #
    #From: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#make_one_shot_iterator
    #Note: The returned iterator will be ***initialized automatically***.
    #A "one-shot" iterator ***does not*** currently support re-initialization.
    #
    train_dataset = tf.data.Dataset.from_tensor_slices(train_data).batch(BATCH_SIZE).repeat()
    train_iter = train_dataset.make_one_shot_iterator()
    train_element = train_iter.get_next()

    test_dataset = tf.data.Dataset.from_tensor_slices(test_data).batch(BATCH_SIZE).repeat()
    test_iter = test_dataset.make_one_shot_iterator()
    test_element = test_iter.get_next()

    with tf.Session() as sess:
        for epoch_no in range(EPOCHS):
            print(epoch_no, end="")
            for batch_no in range(N_BATCHES):
                element = sess.run(train_element)
                training_data_batch = element[0]
                if(0 == epoch_no):
                    assert(BATCH_SIZE == np.size(training_data_batch, 0))
                else:
                    if((epoch_no - 1) == batch_no):
                        assert((DATA_SIZE % BATCH_SIZE) == np.size(training_data_batch, 0))
                        #N_BATCHES (given by DATA_SIZE // BATCH_SIZE) is 78.
                        #78 batches are read every epoch. That leaves 8 slices at the end of the epoch_no 0
                        #Epoch 1 batch_no 0 reads only 8 when the iterator cycles back. In the remaining 77, 
                        #(77 * BATCH_SIZE) = 4928 slices would be read. Leaving (5000 - 4928) 72 slices. 
                        #So Epoch 2 will read 64 slices in batch_no 0 and then 8 slices in batch_no 1 before 
                        #the iterator goes back. So we have a batch of 8 slices rippling forward with each epoch
                        #if (2 == epoch_no):
                    else:
                        assert(BATCH_SIZE == np.size(training_data_batch,0))

            for _ in range(N_BATCHES):
                sess.run(test_element)

@how_much
def initialisable(**kwargs):
    print('\ninitialisable:')
    
    #x and y are placeholders. They have to be loaded. The types and shapes are known
    #not the values.
    #The same placeholder is used both for train and for test data.
    #Hence the graph needs only one dataset node and one iterator
    dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(BATCH_SIZE).repeat()

    iter = dataset.make_initializable_iterator()
    iter_init = iter.initializer
    elements = iter.get_next()

    with tf.Session() as sess:
        for i in range(EPOCHS):
            print(i, end="")
            #Initialize to run dataset node that has been loaded with training data
            sess.run(iter_init, feed_dict={ x: train_data[0], y: train_data[1]})
            for _ in range(N_BATCHES):
                sess.run(elements)
                
            #Re-initialize to run the ***same dataset node*** *** but with test data ***    
            sess.run(iter_init, feed_dict={ x: test_data[0], y: test_data[1]})
            for _ in range(N_BATCHES):
                sess.run(elements)

@how_much            
def reinitializable(**kwargs):
    print('\nreinitializable:')
    # create two datasets, one for training and one for test
    train_dataset = tf.data.Dataset.from_tensor_slices((x,y)).batch(BATCH_SIZE).repeat()
    test_dataset = tf.data.Dataset.from_tensor_slices((x,y)).batch(BATCH_SIZE).repeat()
    
    # create ***an*** iterator of the correct shape and type
    iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
    elements = iter.get_next()
    
    # create the nodes to initialize the iterators to iterate over the train_dataset and the test_dataset
    # See https://www.tensorflow.org/api_docs/python/tf/data/Iterator#make_initializer.
    #The same iterator node (iter) created above is re-initialized
    train_init_op = iter.make_initializer(train_dataset)
    test_init_op = iter.make_initializer(test_dataset)

    with tf.Session() as sess:
        for i in range(EPOCHS):
            print(i, end="")
            #Load the placeholders with train data and train label
            #Iniitialize the iterator to ***iterate over training data***
            sess.run(train_init_op, feed_dict={ x: train_data[0], y: train_data[1]})
            for _ in range(N_BATCHES):
                sess.run(elements)

            #Load the placeholders with test data and test label
            #RE-INITIALIZE the iter to ***now iterate over test data***
            sess.run(test_init_op, feed_dict={ x: test_data[0], y: test_data[1]})
            for _ in range(N_BATCHES):
                sess.run(elements)

@how_much            
def feedable(**kwargs):
    print('\nfeedable:')
    # create two datasets, one for training and one for test
    train_dataset = tf.data.Dataset.from_tensor_slices((x,y)).batch(BATCH_SIZE).repeat()
    test_dataset = tf.data.Dataset.from_tensor_slices((x,y)).batch(BATCH_SIZE).repeat()
    
    # create the iterators from the dataset
    train_iterator = train_dataset.make_initializable_iterator()
    test_iterator = test_dataset.make_initializable_iterator()

    handle = tf.placeholder(tf.string, shape=[])
    
    #See https://www.tensorflow.org/api_docs/python/tf/data/Iterator#from_string_handle
    iter = tf.data.Iterator.from_string_handle(
                            handle, \
                            train_dataset.output_types, \
                            train_dataset.output_shapes)
    elements = iter.get_next()
    
    make_train_string_handle = train_iterator.string_handle()
    make_test_string_handle = test_iterator.string_handle()
    
    init_train_iterator = train_iterator.initializer
    init_test_iterator = test_iterator.initializer

    with tf.Session() as sess:
        #See https://www.tensorflow.org/api_docs/python/tf/data/Iterator#string_handle
        train_string_handle = sess.run(make_train_string_handle)
        test_string_handle = sess.run(make_test_string_handle)

        #See https://www.tensorflow.org/api_docs/python/tf/data/Iterator#initializer
        sess.run(init_train_iterator, feed_dict={ x: train_data[0], y: train_data[1]})
        sess.run(init_test_iterator, feed_dict={ x: test_data[0], y: test_data[1]})

        for i in range(EPOCHS):
            print(i, end="")
            for _ in range(N_BATCHES):
                sess.run(elements, feed_dict={handle: train_string_handle})
            for _ in range(N_BATCHES):
                sess.run(elements, feed_dict={handle: test_string_handle})

print("")
one_shot(log_time=log_time)
print("")
initialisable(log_time=log_time)
print("")
reinitializable(log_time=log_time)
print("")
feedable(log_time=log_time)

sorted((value,key) for (key,value) in log_time.items())